<a href="https://colab.research.google.com/github/sprksh-crypto/MachineLearningProjects/blob/Classification/createSavedDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#I'm trying to access the pickle file through drive, so I'll upload it to drive and then access it that way.
from google.colab import drive

#for authorisation
drive.mount('/content/drive')

#Drive files in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

#Now let's look at the data set.
import pandas as pd

dataset = pd.read_pickle(r'/content/drive/My Drive/appml-assignment1-dataset-v2.pkl')

#Printing the dataset to see:
#dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 appml-assignment1-dataset-v2.pkl   'Essay Drafts'
'Assignment 1 ML.ipynb'		    'ML textbook code 1.ipynb'
 buildAndTrainModel		    'Physics Project'
 correct.tfrecord		    'Physics Scripts'
 createSavedDataset		     Pics
 customImputeLayerDefinition.ipynb   Untitled0.ipynb
 dataset.tfrecord


In [ ]:
#There are two dataframes, x and y. We've been asked to extract these two variables.
dataframeX= dataset ['X']
dataframeY= dataset['y']

#Print X to see.
#dataframeX

In [ ]:
#Now printing Y.
#dataframeY

In [ ]:
#We've been asked in 3.1.1.1 to "use numpy digitize command to quantize the fractional change of the next hour’s CAD-high (in variable y)
#versus the previous hours CAD-close into 23 bins, based on 21 evenly spaced boundaries stretching from −.001
#to .001 and including bins for being below −.001 and above .001."

#we get 22 bins, but this is ok.

import numpy as np

#to create the fractional change of the next hour’s CAD-high (in variable y) versus the previous hours CAD-close.

fractional_change= (dataframeY- dataframeX['CAD-close'])/ dataframeX['CAD-close']

#to create evenly spaced bins based on 21 evenly spaced boundaries from -.001 to .001

evenly_spaced_bins=np.linspace(-.001, .001, num=21)

#Print bins.
#evenly_spaced_bins

In [ ]:
#To see the fractional change.
#fractional_change

In [ ]:
#using numpy digitise to quantise the fractional change.
bin=np.digitize(fractional_change, evenly_spaced_bins)

#to see the frequency within each bin.
#np.bincount(bin)

In [ ]:
#Set bin to a variable called target in dataframeX.
dataframeX['target']=bin

#Print the target variable within the dataframe.
#dataframeX['target']

In [ ]:
#we can count 22 bins to verify.
#another way to see the frequency in each bin.
#import sys

#np.set_printoptions(threshold=sys.maxsize)
#res=pd.DataFrame({'quantized':target})
#res['quantized'].value_counts()

In [ ]:
#For the 3.1.1.2 We've been asked to "create a 7 valued categorical attribute indicating the day of the week associated with the date attribute in the dataframe X".
#by looking at dataframe X, we can see that the dates are in the format YYYY-MM-DD.

#let's look at that data type.
#d= dataframeX['date']
#d

In [ ]:
#it is a "datetime64" type, so this is the solution suggested by the NumPy API references.
dataframeX['weekday']= dataframeX['date'].dt.dayofweek

#let's check
#dataframeX['weekday']

#O is monday, 6 is sunday.

In [ ]:
#Let's check that there are only 7 categories.
#np.bincount(dataframeX['weekday'])

In [ ]:
from os import times_result
#For 3.1.1.3 We've been asked to "create a 24 valued categorical attributed indicatin the hour of the day associated with the date attribute in the dataframe X".
#We know to use "datetime64".

dataframeX['hour']= dataframeX['date'].dt.hour

#Print hour.
#dataframeX['hour']

In [ ]:
#to check that there are 24 categories.

#np.bincount(dataframeX['hour'])

In [ ]:
#For 3.1.1.4 We've been asked to "create a 12 valued categorical attribute indicate the month of the year associated with the date attribute in the dataframe X".
#similar method as above:

dataframeX['month']= dataframeX['date'].dt.month
#Print month.
#dataframeX['month']

In [ ]:
#There should be 12 categories.
#np.bincount(dataframeX['month'])
#There are technically 13, but 0 has none. So, from 1-12, with no values for 0.

In [ ]:
#For 3.1.2 sort 'tickers' as a FloatList. 'weekday', 'hour', 'month', and 'target' as Int64List. Create example protobufs for each instance.
import tensorflow as tf
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

with tf.io.TFRecordWriter('/content/drive/My Drive/dataset.tfrecord') as f:
  for row in dataframeX.iterrows():
    myDict={"tickers": Feature(float_list= FloatList(value=row[1][dataframeX.columns[1:-4]].values)),
            "weekday": Feature(int64_list= Int64List(value=[row[1]['weekday']])),
            "hour": Feature(int64_list= Int64List(value=[row[1]['hour']])),
            "month": Feature(int64_list= Int64List(value=[row[1]['month']])),
            "target": Feature(int64_list= Int64List(value=[row[1]['target']]))}
    myExamp=Example(features=Features(feature=myDict))
    f.write(myExamp.SerializeToString())

#Creating an example protobuf for each instance. Serialising each example object into a string and written to a tfrecords file,
#named dataset.tfrecords containing example protobufs.

In [ ]:
#Now checking if the file created matches the correct.tfrecords file.

#filepath= ["/content/drive/My Drive/correct.tfrecord"]
#filepath1=['/content/drive/My Drive/dataset.tfrecord']

correct_dataset = tf.data.TFRecordDataset(filepath)
#my_dataset = tf.data.TFRecordDataset(filepath1)

#Checking Correct Dataset:
#for raw_record in correct_dataset.take(1):
#  example=tf.train.Example()

#example.ParseFromString(raw_record.numpy())

#print(example)

In [ ]:
#Checking my dataset:
#for raw_record in my_dataset.take(1):
#  example=tf.train.Example()

#example.ParseFromString(raw_record.numpy())

#print(example)

#The difference between the two datasets is clearly within the two target variables, but that has been addressed, 22 bins rather than 23.

In [ ]:
#Another way to check:
#feature_description= {"tickers": tf.io.FixedLenFeature([188], tf.float32, default_value=np.zeros(188)),
#                      "weekday": tf.io.FixedLenFeature([], tf.int64, default_value=0),
#                      "hour": tf.io.FixedLenFeature([], tf.int64, default_value=0),
#                      "month": tf.io.FixedLenFeature([], tf.int64, default_value=0),
#                      "target":tf.io.FixedLenFeature([], tf.int64, default_value=0)}

#for serialized_example in correct_dataset:
#  parsed_example= tf.io.parse_single_example(serialized_example,feature_description)

#for serialized_example in my_dataset:
#  parsed_example1= tf.io.parse_single_example(serialized_example,feature_description)